In [74]:
import config
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

In [75]:
#import config data
mysql = config.mysql

#connect to database
connection_string = (
    f"postgresql://{mysql['user']}:{mysql['password']}@"
    f"{mysql['host']}:{mysql['port']}/{mysql['dbname']}"
)

engine = create_engine(connection_string)

# To create a connection, you can use
conn = engine.connect()
    

In [76]:
#list of provinces in  Poland
provinces = [
    "dolnoslaskie"
    ,"kujawsko_pomorskie"
    ,"lubelskie"
    ,"lubuskie"
    ,"lodzkie"
    ,"malopolskie"
    ,"mazowieckie"
    ,"opolskie"
    ,"podkarpackie"
    ,"podlaskie"
    ,"pomorskie"
    ,"slaskie"
    ,"swietokrzyskie"
    ,"warminsko_mazurskie"
    ,"wielkopolskie"
    ,"zachodniopomorskie"
    ]

In [77]:
#dataframe to province total area renewable solar and wind
solar_area_append_total = {'Province': [], 'solar_area': [], 'wind_area_total': []}
# Import data from database
for province in provinces:
    #select tables
    table_name_solar = f"solar_radiation_photovoltaic_area_{province}"
    table_name_wind = f"wind_speed_area_{province}"
    
    #select data in tables
    query_solar = f"SELECT SUM(area) as solar_total_area FROM {table_name_solar}"
    query_wind = f"SELECT SUM(area) as wind_total_area FROM {table_name_wind}"
    #result to pandas
    solar_result = pd.read_sql(query_solar, conn)
    wind_result = pd.read_sql(query_wind, conn)
    
    #check select data 
    if not solar_result.empty and solar_result['solar_total_area'][0] is not None:
        solar_area = solar_result['solar_total_area'][0]
    else:
        solar_area = 0

    if not wind_result.empty and wind_result['wind_total_area'][0] is not None:
        wind_area = wind_result['wind_total_area'][0]
    else:
        wind_area = 0
    
    #append data to column and change m^2 to km^2
    solar_area_append_total['Province'].append(province)
    solar_area_append_total['solar_area'].append(round(solar_area * 1e-6,2))
    solar_area_append_total['wind_area_total'].append(round(wind_area * 1e-6,2))

df_solar_wind = pd.DataFrame(solar_area_append_total)



ValueError: All arrays must be of the same length

In [ ]:
# Generowanie wykresu słupkowego
plt.figure(figsize=(14, 7))
width = 0.35  # szerokość słupków

# Pozycje słupków dla każdego województwa
ind = range(len(df_solar_wind['Province']))

# Tworzenie słupków dla energii słonecznej i wiatrowej
plt.bar(ind, df_solar_wind['solar_area'], width=width, label='Solar Area', color='blue')
plt.bar([i + width for i in ind], df_solar_wind['wind_area_total'], width=width, label='Wind Area', color='green')

# Dodanie opisów i tytułu
plt.xlabel('Province')
plt.ylabel('Total Area (km²)')
plt.title('Comparison of Solar and Wind Energy Area by Province')
plt.xticks([i + width / 2 for i in ind], df_solar_wind['Province'], rotation=90)
plt.legend()

# Wyświetlenie wykresu
plt.tight_layout()
plt.show()

In [ ]:
df_solar_wind

In [ ]:
#photovoltaic standard panels parametrs [km^2]
height_panel = 1.016
width_panel = 1.686
power_output = 350 #[W]
power_output_m2 = power_output / (height_panel * width_panel)

#panel efficiency by producent 25*C, 1000 [Wm^2]/ power efficency ~20%
efficiency = (power_output / (height_panel*width_panel  * 1000))
efficiency

#power_per_m2 = efficiency_producent_m2 * irradiance


In [ ]:
#create effective area. We can't use whole area. In this case we use 80% area to solar panels and 20% for equipment and maintenance access
#parametrs 
solar_maintenance_access = 0.2
effective_solar_panel_area = 1 - solar_maintenance_access

#calculate the effective area
df_solar_wind['solar_effective_area'] = (df_solar_wind['solar_area'] * effective_solar_panel_area)

In [80]:
#select tables
province = 'dolnoslaskie'
table_name_solar = f"solar_radiation_photovoltaic_area_{province}"

#select data in tables
query_solar = f"SELECT SUM(area * {effective_solar_panel_area} * january) FROM {table_name_solar}"

#result to pandas
aa = pd.read_sql(query_solar, conn)

aa

,sum
0,1.471005e+10
